In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [ ]:
dataset=pd.read_csv('customers.csv', sep=";")

In [ ]:
n=len(dataset)-1 #number of clients

In [ ]:
n

In [ ]:
Q=100 #vehicle capacity

In [ ]:
N = [i for i in range(1, n+1)] #set of clients
V = [0] + N #clients+central depot

In [ ]:
N

In [ ]:
V

In [ ]:
q={i: dataset.iloc[i]['DEMAND'] for i in N} #extraction of customers' demand
#q = {i: rnd.randint(1, 10) for i in N}

In [ ]:
q

In [ ]:
K=range(1,3) #numer of vehicles range(1,4)--> 3 vehicles, range(1,3)--> 2 vehicles...

In [ ]:
xc = dataset['XCOORD'].tolist() #extract locations
yc = dataset['YCOORD'].tolist()

In [ ]:
xc

In [ ]:
yc

In [ ]:
plt.plot(xc[0], yc[0], c='r', marker='s') # central depot location
plt.scatter(xc[1:], yc[1:], c='b') #clients location

In [ ]:
# function to compute distances between nodes
from numpy import sin, cos, arccos, pi, round

def rad2deg(radians):
    degrees = radians * 180 / pi
    return degrees

def deg2rad(degrees):
    radians = degrees * pi / 180
    return radians

def getDistanceBetweenPointsNew(latitude1, longitude1, latitude2, longitude2, unit = 'kilometers'):
    
    theta = longitude1 - longitude2
    
    distance = 60 * 1.1515 * rad2deg(
        arccos(
            (sin(deg2rad(latitude1)) * sin(deg2rad(latitude2))) + 
            (cos(deg2rad(latitude1)) * cos(deg2rad(latitude2)) * cos(deg2rad(theta)))
        )
    )
    
    if unit == 'miles':
        return round(distance, 2)
    if unit == 'kilometers':
        return round(distance * 1.609344, 2)

In [ ]:
A = [(i, j, k) for i in V for j in V if i != j for k in K]
Y = [(i, k) for i in V for k in K]
U = [(i, k) for i in N for k in K]
# definition of variables domain

In [ ]:
c = {(i, j): getDistanceBetweenPointsNew(xc[i], yc[i], xc[j], yc[j], 'kilometers') for i, j, k in A}
 #distance between nodes

In [ ]:
c

In [ ]:
#pip install gurobipy

In [ ]:
from gurobipy import Model, GRB, quicksum

In [ ]:
mdl = Model('CVRP')

In [ ]:
#decision variables definition
x = mdl.addVars(A, vtype=GRB.BINARY) #travelled arcs
z = mdl.addVars(Y, vtype=GRB.BINARY) #assignment of vehicles
u = mdl.addVars(U, vtype=GRB.CONTINUOUS) #sub-tours avoidance

In [ ]:
mdl.modelSense = GRB.MINIMIZE
mdl.setObjective(quicksum(x[i, j, k]*c[i, j] for i, j, k in A))


In [ ]:
mdl.addConstrs(quicksum(x[i, j, k] for j in V for k in K if j != i) == 1 for i in N)
mdl.addConstrs(quicksum(x[i, j, k] for i in V for k in K if i != j) == 1 for j in N)
#Each node is visited by one vehicle. If a vehicle enter the node than 
#it must also leave that node.

mdl.addConstrs(quicksum(z[i, k] for k in K) == 1 for i in N)
#Each node is assigned to exactly one vehicle


mdl.addConstr(quicksum(z[0, k] for k in K) == len(K))
#Each vehicle must starts its route from the central depot

mdl.addConstrs(quicksum(x[i, j, k] for j in V if j != i) - z[i, k] == 0 for i in V for k in K)
#Each node is visited exactly by the planned vehicle.

mdl.addConstrs((x[i, j, k] == 1) >> (u[i, k]+q[j] == u[j, k]) for i, j, k in A if i != 0 and j != 0)
mdl.addConstrs(u[i, k] >= q[i] for i, k in U)
mdl.addConstrs(quicksum(u[i, k] for i in N) <= Q for k in K)
# Subtour elimination and the total quantity delivered by the vehicle on day t 
#must not exceed the vehicle’s capacity C.

mdl.addConstrs(quicksum((x[j, i, k]-x[i, h, k]) for j in V for h in V if j != h if j != i if i != h) == 0 for i in V for k in K)
#flow conservation constraint


In [ ]:
#mdl.Params.MIPGap = 0.5
mdl.Params.TimeLimit = 60  # optimisation duration in seconds
mdl.optimize()

In [ ]:
active_arcs = [a for a in A if x[a].x > 0.99] #travelled arcs

In [ ]:
active_arcs

In [ ]:
for i, j, k in active_arcs:
    plt.plot([xc[i], xc[j]], [yc[i], yc[j]], c='g', zorder=0)
plt.plot(xc[0], yc[0], c='r', marker='s')
plt.scatter(xc[1:], yc[1:], c='b')

In [ ]:
#time required to travel each route, assuming an average speed of 70km/h
#total_TT={k:quicksum(c[i,j]/70 for i, j, k in active_arcs) for k in K} #

In [ ]:
#total_TT